# A walk in the park - the sourcing, cleaning and processing edition

This is a companion notebook to "A walk in the park" (link here) detailing all the major steps and tools in sourcing and cleaning the data to make it suitable for visualisation and analysis. This notebook is structure in a similar way, broken up into the 3 major questions.

1. [Data](#data)
1. [How this notebook works](#howthis)
1. [Initial setup, sourcing, and cleaning](#setup)
1. [Question 1: Are there any significant differences between teams, looking only at the game stats? ](#question-1)
1. [Question 2: Does the average temperature affect team performance?](#question-2)
1. [Question 3: Is there a different body type between professional codes?](#question-3)
1. [Data sources](#data-sources)
1. [Installed packages and technology](#packages)


### <a id="data">Data:</a>
Some of the raw datafiles will be made available in the repo. For other data (football players, soccer, NFL, NRL) you will need to run the sourcing/scraping scripts to download the data.

### <a id="howthis">How this notebook works:</a>
This notebook assumes a moderate working knowledge of R, and as such I won't be parsing each line in my code, but pulling out 'highlights' for discussion. Full code is available in the git repo here.

### <a id="setup">Initial setup, sourcing, and cleaning:</a>

Setting up R project:
Taking inspiration from "Nice R Code" ([link][1]), I initially created 3 R files:
1. **analysis-function.R** - containing all the the major analysis code. The top of the file looks like this:
```
library(RCurl)
library(stringr)
library(plyr)
library(dplyr)
...
setwd("/Users/alex/Documents/datasets/afl")
source("src/data_clean.R")
setwd("/Users/alex/Documents/datasets/afl")
source("src/functions.R")
```
This enables me to quickly run all three files by executing (or 'sourcing') the first file. Centralising the list of required packages also enables me to track and therefore easily upgrade the packages when required (R went through a major update since I started this project, as did ggplot. So I had to re-install all packages twice)<br><br>
2. **data_clean.R** - contains all the code required to clean the data into dataframes for the other scripts to use. <br><br>
3. **functions.R** - contains all the functions used in analysis code. Some of them required to generate multiple plots and setup global variables.

I also created the following file **plotlyscript.R**, as I was using principally ggplot and base R libraries to plot the data, and I wanted to separate my plotly code from the rest of the project.


#### Directory structure:
```
afl/
|── src/
|── data/
|── figs/
|── output/
|── references/
```

* The ``src`` directory contains all the R, Python and bash scripts used in the project
* The ``data`` directory contains all the raw data used in this project, including downloaded webpages, csvs and databases. This directory is used as 'read-only' to preserve the integrity of the data
* The ``figs`` directory where all saved plots and charts are stored
* The ``output`` directory where any processed data and dataframes are stored
* The ``references`` directory where I downloaded reference JPEGs for AFL team logos and colours


#### Technology stack:
See platform and packages here: [Installed packages and technology](#packages)

[1]: https://nicercode.github.io/blog/2013-04-05-projects/ "Nice R code"

## <a id="question-1"> Question 1: Are there any significant differences between teams, looking only at the game stats?</a>

Using R to download the file that will be used for the main data set I run:
```
download.file("http://www.aussportsbetting.com/historical_data/afl.xlsx","data/raw/afl.xlsx",method="curl")
```
In subsequent re-runs of the data_clean script I would comment that line out, and use the already downloaded xlsx  (Ie I don't need to download it each time).

Using ``in2csv`` from the bash ``csvkit`` toolset, I convert the afl.xlsx into a csv file. 
```
system("in2csv data/raw/afl.xlsx > data/raw/afl.csv")
```
*I find that to create R dataframes, CSVs are the best file format to use. Also, if I need to run any further processing in bash or python, CSVs are the better option - thus I did not consider read.xlsx or similar R methods*

As the script/project grows, the duration of the file and data processing extends, so I add console feedback to call out completion of some sections. 

To time the data cleaning process I also enclose the code as follows:
```
a1 <- Sys.time()
...
...
a2 <- Sys.time()
print(a2 -a1)
print("functions.R & data_clean.R sourcing complete") 
```
R automatically works out the human readable time with this line ``print(a2-a1)`` and prints out something like
```
> print(a2 -a1)
Time difference of 5.701957 mins
```
The first analysis required a comparison between goals and behinds. In Aussie rules footbal (AFL), a goal is worth 6 points and a behind is worth a single point, however for this analysis the total number of points scored was not important, I wanted to compare frequency of goals and behinds kicked. 

Whilst the dataset does contain the following variables: "Home.Goals","Home.Behinds","Away.Goals","Away.Behinds", I need to extract the counts by team. I do this by:
1. create a vector ``teams`` with a unique list of all the teams, and then 
1. expand that to two vectors that include "[Team].Goals", and "[Team].Behinds" for all teams.
1. use each vector to create new variables in the dataset
1. fill with NAs
   
Code snippet:

```
teams <- unique(tmp.afl$Away.Team)
teams <- gsub(" ",".", teams)

teams.goals <- paste(teams, ".Goals", sep="")
teams.behinds <- paste(teams, ".Behinds", sep="")

tmp.afl[,teams.goals] <- NA
tmp.afl[,teams.behinds] <- NA
```

Now populate the new variables using a loop that:
1. First identifies all rows with a specific team name
1. Then identifies which variable (column) in the data set has the same name
1. Populates the column with the data from the respective "Home.Goals" and "Home.Behinds" columns
1. Wash, rinse repeat with a second loop that does exactly the same with "Away.Goals" and "Away.Behinds"

Code snippet:

```
for ( teamname in unique(tmp.afl$Home.Team) ) {
  
  RowIndex = which(tmp.afl$Home.Team == teamname)
  # find ncol index of appropriate columns
  ColIndex = which( substr(names(tmp.afl),1,nchar(teamname)) %in% gsub(" ",".", teamname))
  
  # Home Goals
  tmp.afl[RowIndex, ColIndex[1]] <- tmp.afl[RowIndex,9 ]
  # Home Behinds
  tmp.afl[RowIndex, ColIndex[2]] <- tmp.afl[RowIndex,10 ]
  
}
```
I experimented with using ``ldply`` and other ``plyr`` methods, but for the sake of readability and easier debugging, I used a ``for`` loop. 






## <a id="question-2">Question 2: Does the average temperature affect team performance?</a>
#### (i.e do some teams play better at certain temperatures?)


#### Method:
**Sourcing the data:**
1. Detail the physical location of each stadium using wikipedia.
2. Using bom.gov.au and google maps, locate the weather stations closest to each stadium
3. Download the weather records from selected weather stations (http://www.bom.gov.au/climate/data/index.shtml?bookmark=201)
4. Some of the weather records are incomplete, so work out secondary stations that are still near the stadiums that can fill in the gaps

**Cleaning the data**
<i> Note: not all steps for data cleaning are detailed, just highlights. Full code available from the repo: </i>

Assuming you have extracted all the bom zip files csvs into a single driectory:
1. Create a data frame with all weather records (ldply is our friend):
```
lstofCSVs <- as.list(list.files("data/raw/bomdata"))
tmpcsv <- ldply(paste("data/raw/bomdata/",lstofCSVs,sep=""), read.csv)
```
```
head(tmpcsv[which(tmpcsv$Date>'2009-10-01'),], 2)
      Product.code Bureau.of.Meteorology.station.number Year Month Day
15616   IDCJAC0009                                 9151 2009    10   2
15617   IDCJAC0009                                 9151 2009    10   3
      Rainfall.amount..millimetres.
15616                           0.0
15617                           2.2
      Period.over.which.rainfall.was.measured..days. Quality
15616                                             NA       Y
15617                                              1       Y
      Maximum.temperature..Degree.C.
15616                             NA
15617                             NA
      Days.of.accumulation.of.maximum.temperature
15616                                          NA
15617                                          NA
      Minimum.temperature..Degree.C.
15616                             NA
15617                             NA
      Days.of.accumulation.of.minimum.temperature       Date
15616                                          NA 2009-10-02
15617                                          NA 2009-10-03
```


1. Create a ISO 8601 formatted date field (helps with dataframe operations later on)<br><br>
``tmpcsv$Date <- as.Date(paste(tmpcsv$Year, "-", sprintf("%02d", tmpcsv$Month),
"-",sprintf("%02d",tmpcsv$Day), sep=""))
``<br><br>

1. Get list of dates of all games played (2016 only)<br><br>
``lstDatesofGamesPlayed <- unique(tmp.afl[which(tmp.afl$Date < "2017-01-01"),]$Date)``<br><br>

1. Create a frequency table of all city climate records <br><br>
``vecCities <- sapply(split(lstofdftemp.dates, lstofdftemp.dates$City),nrow)``<br><br>

1. loop through cities and where the frequency count is less than the number of dates of games played, get the records from secondary priority stations, and then collapse the dataframe to get rid of NAs<br>

    ``df.temp.missing <- setDT(df.temp.missing)[,lapply(.SD, na.omit), by=Date]``



## <a id="question-3">Question 3: Is there a different body type between professional codes?</a>
#### (and how different?)

First source the appropriate AFL player stats from here (http://afltables.com/afl/stats/playersA_idx.html), using a python script to download the pages using ``urllib.request`` and to parse them using ``BeautifulSoup``. I initially attempted to use ``scrapy`` but found it too convoluted for what I wanted to achieve. 

For each iteration succesfull iteration of the loop, there is a ``time.sleep(4)`` which pauses the program execution for 4 seconds before looping again. I do this as a matter of courtesy  to those that run the website.

1. select player pages whose first year is >= 2009 (start of AFL fixtures dataset) or the last year played >= 2009.






#### <a id="packages">Installed packages and technology stack</a>

R Session Info:
```
R version 3.4.0 (2017-04-21)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: OS X El Capitan 10.11.6

Matrix products: default
BLAS: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.4/Resources/lib/libRlapack.dylib

locale:
[1] en_AU.UTF-8/en_AU.UTF-8/en_AU.UTF-8/C/en_AU.UTF-8/en_AU.UTF-8

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] webshot_0.4.0       htmltools_0.3.6     knitr_1.15.1       
 [4] pander_0.6.0        readr_1.1.0         RColorBrewer_1.1-2 
 [7] plotly_4.6.0        formattable_0.2.0.1 DBI_0.6-1          
[10] gridExtra_2.2.1     rvest_0.3.2         xml2_1.1.1         
[13] XML_3.98-1.6        data.table_1.10.4   reshape_0.8.6      
[16] reshape2_1.4.2      ggplot2_2.2.1       plyr_1.8.4         
[19] stringr_1.2.0       RCurl_1.95-4.8      bitops_1.0-6       
[22] dplyr_0.5.0         RSQLite_1.1-2      

Notebook:

IRdisplay    "0.4.4"           
IRkernel     "0.8.6.9000

Python version: Python 3.5.2 |
Anaconda 4.2.0 (x86_64)
csvkit 1.0.2|
```

